In [68]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
from scipy.optimize import minimize

In [69]:
# Load data
portfolio = pd.read_csv("initial_portfolio.csv")
rf = pd.read_csv("rf.csv")
prices = pd.read_csv("DailyPrices.csv")


In [70]:
#part 1
#option 1 for risk free rate

def return_calculate(prices, method="DISCRETE", date_column="Date"):

    vars = prices.columns.tolist()
    vars = [v for v in vars if v != date_column]
    
    p = prices[vars].values
    n = p.shape[0]
    m = p.shape[1]
    p2 = np.zeros((n-1, m))
    
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]
    
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)

    dates = prices[date_column].iloc[1:n].reset_index(drop=True)
    out = pd.DataFrame({date_column: dates})
    
    for i, var in enumerate(vars):
        out[var] = p2[:, i]
    
    return out

def expost_factor(w, realized_returns, realized_spy, betas): # realized_returns, realized_spy

    _realized_spy = realized_spy.copy()
    stocks = realized_returns.columns.tolist()
    factors = realized_spy.columns.tolist()
    
    n = len(realized_returns)
    m = len(stocks)
    
    p_return = np.zeros(n)
    resid_return = np.zeros(n)
    weights = np.zeros((n, len(w)))
    factor_weights = np.zeros((n, len(factors)))
    last_w = w.copy()
    
    mat_returns = realized_returns[stocks].values
    ff_returns = realized_spy[factors].values

    resid_individual = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            resid_individual[i, j] = mat_returns[i, j] - betas[j] * ff_returns[i, 0]
    
    for i in range(n):
        weights[i, :] = last_w
        for f in range(len(factors)):
            factor_weights[i, f] = np.sum(betas * last_w)
        last_w = last_w * (1.0 + mat_returns[i, :])
        p_r = np.sum(last_w)
        last_w = last_w / p_r
        p_return[i] = p_r - 1
        resid_return[i] = (p_r - 1) - np.dot(factor_weights[i, :], ff_returns[i, :])
    
    _realized_spy['Alpha'] = resid_return
    _realized_spy['Portfolio'] = p_return
    
    total_ret = np.exp(np.sum(np.log(p_return + 1))) - 1
    
    k = np.log(total_ret + 1) / total_ret if total_ret != 0 else 1.0
    
    carino_k = np.zeros_like(p_return)
    for i in range(len(p_return)):
        if p_return[i] != 0:
            carino_k[i] = np.log(1.0 + p_return[i]) / p_return[i] / k
        else:
            carino_k[i] = 1.0 / k
    attrib = pd.DataFrame(ff_returns * factor_weights * carino_k.reshape(-1, 1), columns=factors)
    attrib['Alpha'] = resid_return * carino_k
    for i in range(n):
        for j in range(m):
            resid_individual[i, j] *= weights[i, j]
    attribution = pd.DataFrame({'Value': ['TotalReturn', 'Return Attribution']})
    new_factors = factors + ['Alpha']
    for s in new_factors + ['Portfolio']:
        tr = np.exp(np.sum(np.log(_realized_spy[s] + 1))) - 1
        atr = tr if s == 'Portfolio' else np.sum(attrib[s])
        attribution[s] = [tr, atr]
    Y = np.hstack((ff_returns * factor_weights, resid_return.reshape(-1, 1)))
    X = np.column_stack((np.ones(len(p_return)), p_return))
    B = np.linalg.inv(X.T @ X) @ X.T @ Y
    B = B[1, :]
    cSD = B * np.std(p_return)
    vol_attrib = pd.DataFrame({'Value': ['Vol Attribution']})
    portfolio_vol = 0
    for i, factor in enumerate(new_factors):
        portfolio_vol += cSD[i]
        vol_attrib[factor] = [cSD[i]]
    vol_attrib['Portfolio'] = [portfolio_vol]
    attribution = pd.concat([attribution, vol_attrib], ignore_index=True)
    return {
        'attribution': attribution,
        'weights': weights,
        'factor_weights': factor_weights,
        'resid_individual': resid_individual,
        'resid_return': resid_return,
        'carino_k': carino_k
    }

def run_attribution(realized_returns, realized_spy, last_date, start_prices, portfolio, betas, portfolio_type="Initial"):

    stocks = portfolio['Symbol'].tolist()
    
    holdings = portfolio['Holding'].values
    start_price_values = start_prices.values[0] 
    
    t_value = np.sum(start_price_values * holdings)
    w = (start_price_values * holdings) / t_value
    
    attrib = expost_factor(w, realized_returns, realized_spy, betas)
    print(f"\n{portfolio_type} Total Portfolio Attribution")
    print(attrib['attribution'])
    
    all_results = {
        'Total': attrib['attribution']
    }
    
    portfolios = sorted(list(set(portfolio['Portfolio'])))
    for p in portfolios:
        stocks_mask = portfolio['Portfolio'] == p
        p_stocks = portfolio.loc[stocks_mask, 'Symbol'].tolist()
        
        stock_returns = realized_returns[p_stocks]
        p_start_prices = start_prices[p_stocks].values[0]
        
        p_holdings = portfolio.loc[stocks_mask, 'Holding'].values
        
        p_t_value = np.sum(p_start_prices * p_holdings)
        p_w = (p_start_prices * p_holdings) / p_t_value
        
        p_betas = np.array([betas[stocks.index(s)] for s in p_stocks])

        p_attrib = expost_factor(p_w, stock_returns, realized_spy, p_betas)
        print(f"{portfolio_type} {p} Portfolio Attribution")
        print(p_attrib['attribution'], '\n')
        
        all_results[p] = p_attrib['attribution']
    
    return all_results

def calculate_expected_returns(betas, avg_mkt_return, avg_rf):

    return avg_rf + betas * (avg_mkt_return - avg_rf)

def calculate_covariance_matrix(returns, betas, market_returns):

    n = len(betas)
    market_var = np.var(market_returns)
    
    resid = np.zeros((len(returns), n))
    for i in range(n):
        resid[:, i] = returns.iloc[:, i] - betas[i] * market_returns
    
    resid_var = np.var(resid, axis=0)

    cov_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                cov_matrix[i, j] = betas[i]**2 * market_var + resid_var[i]
            else:
                cov_matrix[i, j] = betas[i] * betas[j] * market_var
    
    return cov_matrix


# Prepare data
prices['Date'] = pd.to_datetime(prices['Date'])
rf['Date'] = pd.to_datetime(rf['Date'])
returns = return_calculate(prices)  # Using our own return_calculate function
returns = pd.merge(returns, rf, on='Date', how='left').ffill()

# Estimate Betas using OLS
stocks = portfolio['Symbol'].tolist()
betas_dict = {}
sigma_epsilons = {}

for stock in stocks:
    subset = returns[returns['Date'] < datetime(2023, 12, 31)]
    Y = subset[stock] - subset['rf']
    X = subset['SPY'] - subset['rf']
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    betas_dict[stock] = list(model.params)[1]  # Index 1 for the slope coefficient
    sigma_epsilons[stock] = model.resid.std()

# Convert dictionary to list in the same order as stocks
betas = np.array([betas_dict[stock] for stock in stocks])

# Realized returns post-2024
realized_returns = returns[returns['Date'] >= datetime(2024, 1, 1)][stocks]
realized_spy = pd.DataFrame({'SPY': returns[returns['Date'] >= datetime(2024, 1, 1)]['SPY']})

# Get start prices on last trading day of 2023
last_date = returns[returns['Date'] < datetime(2024, 1, 1)]['Date'].max()
start_prices = prices[prices['Date'] == last_date][stocks]

# Run initial attribution analysis
initial_results = run_attribution(realized_returns, realized_spy, last_date, start_prices, portfolio, betas, "Initial")


Initial Total Portfolio Attribution
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.035969   0.204731
1  Return Attribution  0.244039 -0.039309   0.204731
2     Vol Attribution  0.007207 -0.000131   0.007076
Initial A Portfolio Attribution
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.095555   0.136642
1  Return Attribution  0.242621 -0.105980   0.136642
2     Vol Attribution  0.007056  0.000348   0.007404 

Initial B Portfolio Attribution
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.028626   0.203526
1  Return Attribution  0.234259 -0.030733   0.203526
2     Vol Attribution  0.006411  0.000442   0.006854 

Initial C Portfolio Attribution
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.022337   0.281172
1  Return Attribution  0.255627  0.025546   0.281172
2     Vol Attribution  0.007230  0.000678   0.007908 



/var/folders/30/4j0_qfq56ll_fl0_jwr614tc0000gn/T/ipykernel_70828/1658277011.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[var] = p2[:, i]


In [71]:
#part 2
import numpy as np
import pandas as pd

# Pre-2024 returns for estimating expected returns and covariance
pre2024_returns = returns[returns['Date'] < '2024-01-01'].copy()
pre2024_rf = rf[rf['Date'] < '2024-01-01']['rf']
pre2024_market = pre2024_returns['SPY']

expected_market_return = pre2024_market.mean()
expected_rf = pre2024_rf.mean()

# Loop through each sub-portfolio
sub_portfolios = portfolio['Portfolio'].unique()

for pf_name in sub_portfolios:
    print(f"\nPortfolio {pf_name} ")

    pf_symbols = portfolio[portfolio['Portfolio'] == pf_name]['Symbol'].tolist()

    # Estimate CAPM expected returns (alpha = 0, Option 1)
    expected_returns_capm = {}
    for symbol in pf_symbols:
        beta = betas_dict[symbol]
        expected_return = expected_rf + beta * (expected_market_return - expected_rf)
        expected_returns_capm[symbol] = expected_return

    mu_vec = np.array([expected_returns_capm[s] for s in pf_symbols])
    excess_mu = mu_vec - expected_rf
    cov_matrix = pre2024_returns[pf_symbols].cov()
    Sigma = cov_matrix.values

    # Unconstrained maximum Sharpe ratio weights
    w_unnormalized = np.linalg.inv(Sigma) @ excess_mu
    w_optimal = w_unnormalized / np.sum(w_unnormalized)
    weights_dict = dict(zip(pf_symbols, w_optimal))

    # Return attribution using Option 1 during 2024
    returns_2024 = returns[returns['Date'] >= '2024-01-01']
    market_2024 = returns_2024['SPY']

    total_sys = 0
    total_idio = 0
    total_ret = 0

    for symbol in pf_symbols:
        Ri = returns_2024[symbol]
        Rm = market_2024
        beta = betas_dict[symbol]

        SR = beta * Rm  # Systematic return (Option 1)
        IR = Ri - SR

        weight = w_optimal[pf_symbols.index(symbol)]
        total_ret += weight * Ri.sum()
        total_sys += weight * SR.sum()
        total_idio += weight * IR.sum()

    # Daily Sharpe ratio calculation
    returns_matrix = returns_2024[pf_symbols].values
    weights_vec = np.array([weights_dict[s] for s in pf_symbols])
    portfolio_daily_returns = returns_matrix @ weights_vec

    daily_mean_return = portfolio_daily_returns.mean()
    portfolio_std = portfolio_daily_returns.std()
    sharpe_ratio = (daily_mean_return - expected_rf) / portfolio_std  # daily Sharpe

    # Attribution table (daily vol-based)
    attribution_table = pd.DataFrame({
        'Value': ['TotalReturn', 'Return Attribution', 'Vol Attribution'],
        'SPY': [total_sys, total_sys, portfolio_std * (total_sys / total_ret)],
        'Alpha': [total_idio, total_idio, portfolio_std * (total_idio / total_ret)],
        'Portfolio': [total_ret, total_ret, portfolio_std]
    })

    print(f"Optimal Sharpe Ratio Portfolio Attribution (Option 1, Unconstrained) — Portfolio {pf_name}")
    print(f"Sharpe Ratio (Daily): {sharpe_ratio:.4f}")
    print(attribution_table)



Portfolio A 
Optimal Sharpe Ratio Portfolio Attribution (Option 1, Unconstrained) — Portfolio A
Sharpe Ratio (Daily): 0.1009
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.243260 -0.003874   0.239386
1  Return Attribution  0.243260 -0.003874   0.239386
2     Vol Attribution  0.007503 -0.000119   0.007384

Portfolio B 
Optimal Sharpe Ratio Portfolio Attribution (Option 1, Unconstrained) — Portfolio B
Sharpe Ratio (Daily): 0.1028
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.241983 -0.000997   0.240986
1  Return Attribution  0.241983 -0.000997   0.240986
2     Vol Attribution  0.007338 -0.000030   0.007308

Portfolio C 
Optimal Sharpe Ratio Portfolio Attribution (Option 1, Unconstrained) — Portfolio C
Sharpe Ratio (Daily): 0.1135
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.244346  0.035776   0.280123
1  Return Attribution  0.244346  0.035776   0.280123
2     Vol Attribution  0.006962  0.0010

In [72]:
#part 4
import numpy as np
import pandas as pd
from scipy.stats import norm, skewnorm, norminvgauss, t as t_dist

# Filter returns before 2024
pre2024_returns = returns[returns['Date'] < '2024-01-01'].copy()
pre2024_returns.set_index('Date', inplace=True)

fit_results = {}

for symbol in stocks:
    data = pre2024_returns[symbol].dropna().values
    data = data - data.mean()  # Set mean ≈ 0

    dist_names = ['Normal', 'Generalized T', 'NIG', 'Skew Normal']
    aic_scores = {}
    fitted_params = {}

    n = len(data)

    # Normal
    try:
        mu, std = norm.fit(data)
        ll = np.sum(norm.logpdf(data, loc=0, scale=std))  # Fix mean = 0
        k = 1
        aic_scores['Normal'] = 2 * k - 2 * ll
        fitted_params['Normal'] = {'std': std}
    except:
        aic_scores['Normal'] = np.inf
        fitted_params['Normal'] = None

    # Generalized T (using scipy.stats.t)
    try:
        df, loc, scale = t_dist.fit(data, floc=0)
        ll = np.sum(t_dist.logpdf(data, df, loc=0, scale=scale))
        k = 2
        aic_scores['Generalized T'] = 2 * k - 2 * ll
        fitted_params['Generalized T'] = {'df': df, 'scale': scale}
    except:
        aic_scores['Generalized T'] = np.inf
        fitted_params['Generalized T'] = None

    # NIG
    try:
        a, b, loc, scale = norminvgauss.fit(data, floc=0)
        ll = np.sum(norminvgauss.logpdf(data, a, b, loc=0, scale=scale))
        k = 3
        aic_scores['NIG'] = 2 * k - 2 * ll
        fitted_params['NIG'] = {'a': a, 'b': b, 'scale': scale}
    except:
        aic_scores['NIG'] = np.inf
        fitted_params['NIG'] = None

    # Skew Normal
    try:
        a, loc, scale = skewnorm.fit(data, floc=0)
        ll = np.sum(skewnorm.logpdf(data, a, loc=0, scale=scale))
        k = 2
        aic_scores['Skew Normal'] = 2 * k - 2 * ll
        fitted_params['Skew Normal'] = {'a': a, 'scale': scale}
    except:
        aic_scores['Skew Normal'] = np.inf
        fitted_params['Skew Normal'] = None

    # Determine best model by AIC
    best_model = min(aic_scores, key=aic_scores.get)
    fit_results[symbol] = {
        'Best Fit': best_model,
        'Params': fitted_params[best_model],
        'AIC': aic_scores[best_model]
    }

# Convert to DataFrame
fit_df = pd.DataFrame(fit_results).T
fit_df.index.name = 'Symbol'

# Display fit results
display(fit_df)

# Count how many stocks selected each best-fit model
fit_summary = fit_df['Best Fit'].value_counts().rename_axis('Distribution').reset_index(name='Stock Count')

print("\nBest Fit Distribution Summary (Stock Count):")
display(fit_summary)



,Best Fit,Params,AIC
Symbol,,,
WFC,Generalized T,"{'df': 5.001478131472827, 'scale': 0.013690793...",-1322.469459
ETN,Generalized T,"{'df': 3.9195527205129665, 'scale': 0.01206865...",-1355.429155
AMZN,Generalized T,"{'df': 5.954095317120535, 'scale': 0.016917840...",-1234.173899
QCOM,Generalized T,"{'df': 5.228936791555922, 'scale': 0.015619414...",-1261.503045
LMT,Generalized T,"{'df': 3.7066212344259615, 'scale': 0.00739510...",-1591.359489
...,...,...,...
MSFT,Generalized T,"{'df': 7.822557002480616, 'scale': 0.013629964...",-1363.007912
PEP,Generalized T,"{'df': 5.81887209927161, 'scale': 0.0076155867...",-1629.584382
CB,Generalized T,"{'df': 5.697500393199872, 'scale': 0.010327721...",-1475.924947



Best Fit Distribution Summary (Stock Count):


,Distribution,Stock Count
0,Generalized T,97
1,NIG,2


In [73]:

from scipy.stats import norm, t as t_dist, skewnorm, norminvgauss, multivariate_normal

# Prepare price dataframe
prices['Date'] = pd.to_datetime(prices['Date'])
price_df = prices.set_index('Date')

# Extract symbol list from portfolio
stocks = portfolio['Symbol'].unique().tolist()

# Compute daily log returns before 2024
log_returns = np.log(price_df[stocks] / price_df[stocks].shift(1)).dropna()
pre2024_returns = log_returns[log_returns.index < '2024-01-01']

# Convert historical returns to CDF values (marginals) using best-fit distribution
u_data = pd.DataFrame(columns=stocks)

for stock in stocks:
    dist_name = fit_df.loc[stock, 'Best Fit']
    params = fit_df.loc[stock, 'Params']
    x = pre2024_returns[stock].dropna()

    if dist_name == 'Normal':
        u_data[stock] = norm.cdf(x, loc=0, scale=params['std'])
    elif dist_name == 'Generalized T':
        u_data[stock] = t_dist.cdf(x, df=params['df'], loc=0, scale=params['scale'])
    elif dist_name == 'Skew Normal':
        u_data[stock] = skewnorm.cdf(x, a=params['a'], loc=0, scale=params['scale'])
    elif dist_name == 'NIG':
        u_data[stock] = norminvgauss.cdf(x, a=params['a'], b=params['b'], loc=0, scale=params['scale'])

# Fit Gaussian copula using correlation of uniform marginals
R = u_data.corr()
n_sim = 1000
copula = multivariate_normal(mean=np.zeros(len(stocks)), cov=R)
sim_u = pd.DataFrame(norm.cdf(copula.rvs(size=n_sim)), columns=stocks)

# Invert the CDF to obtain simulated returns from uniform samples
simulated_returns = pd.DataFrame(columns=stocks)

for stock in stocks:
    dist_name = fit_df.loc[stock, 'Best Fit']
    params = fit_df.loc[stock, 'Params']
    u = np.clip(sim_u[stock], 1e-6, 1 - 1e-6)

    if dist_name == 'Normal':
        simulated_returns[stock] = norm.ppf(u, loc=0, scale=params['std'])
    elif dist_name == 'Generalized T':
        simulated_returns[stock] = t_dist.ppf(u, df=params['df'], loc=0, scale=params['scale'])
    elif dist_name == 'Skew Normal':
        simulated_returns[stock] = skewnorm.ppf(u, a=params['a'], loc=0, scale=params['scale'])
    elif dist_name == 'NIG':
        simulated_returns[stock] = norminvgauss.ppf(u, a=params['a'], b=params['b'], loc=0, scale=params['scale'])

# Simulate returns from multivariate normal (MVN) as baseline
mean_vec = pre2024_returns[stocks].mean().values
cov_mat = pre2024_returns[stocks].cov().values
mvn_samples = np.random.multivariate_normal(mean_vec, cov_mat, size=n_sim)
mvn_df = pd.DataFrame(mvn_samples, columns=stocks)

# Compute 1-day VaR and ES for each portfolio and for the total portfolio
portfolio_results = []

for name in ['A', 'B', 'C', 'Total']:
    if name != 'Total':
        sub = portfolio[portfolio['Portfolio'] == name].set_index('Symbol')
    else:
        sub = portfolio.set_index('Symbol')

    holdings = sub['Holding']
    prices_at_start = price_df.loc['2023-12-29', holdings.index]
    values = holdings * prices_at_start
    weights = values / values.sum()
    valid_stocks = [s for s in stocks if s in weights.index]
    w_vec = weights.reindex(valid_stocks).fillna(0).values

    sim_port_returns = simulated_returns[valid_stocks].dot(w_vec)
    mvn_port_returns = mvn_df[valid_stocks].dot(w_vec)

    VaR_95_cop = np.percentile(sim_port_returns, 5)
    ES_95_cop = sim_port_returns[sim_port_returns <= VaR_95_cop].mean()
    VaR_95_mvn = np.percentile(mvn_port_returns, 5)
    ES_95_mvn = mvn_port_returns[mvn_port_returns <= VaR_95_mvn].mean()

    port_value = values.sum()
    portfolio_results.append({
        'Portfolio': name,
        'VaR_95 (Copula $)': round(abs(VaR_95_cop * port_value), 2),
        'ES_95 (Copula $)': round(abs(ES_95_cop * port_value), 2),
        'VaR_95 (MVN $)': round(abs(VaR_95_mvn * port_value), 2),
        'ES_95 (MVN $)': round(abs(ES_95_mvn * port_value), 2),
        'VaR_95 (Copula %)': round(abs(VaR_95_cop) * 100, 2),
        'ES_95 (Copula %)': round(abs(ES_95_cop) * 100, 2),
        'VaR_95 (MVN %)': round(abs(VaR_95_mvn) * 100, 2),
        'ES_95 (MVN %)': round(abs(ES_95_mvn) * 100, 2)
    })

# Output results
var_es_summary = pd.DataFrame(portfolio_results)
print("\nPortfolio 1-day VaR and ES (Copula vs MVN):")
print(var_es_summary.to_string(index=False))



Portfolio 1-day VaR and ES (Copula vs MVN):
Portfolio  VaR_95 (Copula $)  ES_95 (Copula $)  VaR_95 (MVN $)  ES_95 (MVN $)  VaR_95 (Copula %)  ES_95 (Copula %)  VaR_95 (MVN %)  ES_95 (MVN %)
        A            4174.03           6001.28         4112.91        5183.07               1.41              2.03            1.39           1.75
        B            3836.07           5311.78         3590.75        4531.93               1.37              1.89            1.28           1.61
        C            4120.11           5572.58         3470.12        4353.63               1.54              2.08            1.30           1.63
    Total           11741.71          16453.37        10805.91       13826.27               1.39              1.95            1.28           1.64


In [74]:
#part 5
from scipy.optimize import minimize

def compute_es_per_stock(simulated_returns, alpha=0.05):
    es_dict = {}
    for symbol in simulated_returns.columns:
        data = simulated_returns[symbol].dropna()
        var = np.percentile(data, alpha * 100)
        es = data[data <= var].mean()
        es_dict[symbol] = abs(es)  # Use absolute ES as risk
    return es_dict

def get_risk_parity_weights(es_dict):
    assets = list(es_dict.keys())
    es_values = np.array([es_dict[a] for a in assets])

    def objective(w):
        total_risk = np.dot(w, es_values)
        risk_contrib = w * es_values
        avg_risk = total_risk / len(w)
        return np.sum((risk_contrib - avg_risk) ** 2)

    n = len(assets)
    w0 = np.ones(n) / n
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1)] * n

    result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    weights = result.x
    return dict(zip(assets, weights))

#  for each sub-portfolio 
risk_parity_weights = {}

for name in ['A', 'B', 'C']:
    print(f"\nRisk Parity for Portfolio {name} ")

    sub = portfolio[portfolio['Portfolio'] == name].set_index('Symbol')
    sub_stocks = sub.index.tolist()
    
    # calculate stock-level ES
    es_dict = compute_es_per_stock(simulated_returns[sub_stocks], alpha=0.05)
    
    # optimize for risk parity weights
    weights = get_risk_parity_weights(es_dict)
    risk_parity_weights[name] = weights

    # Display result
    print("Risk Parity Weights (based on ES):")
    for k, v in weights.items():
        print(f"{k}: {v:.4f}")



Risk Parity for Portfolio A 
Risk Parity Weights (based on ES):
WFC: 0.0303
ETN: 0.0303
AMZN: 0.0303
QCOM: 0.0303
LMT: 0.0303
KO: 0.0303
JNJ: 0.0303
ISRG: 0.0303
XOM: 0.0303
MDT: 0.0303
DHR: 0.0303
PLD: 0.0303
BA: 0.0303
PG: 0.0303
MRK: 0.0303
AMD: 0.0303
BX: 0.0303
PM: 0.0303
SCHW: 0.0303
VZ: 0.0303
COP: 0.0303
ADI: 0.0303
BAC: 0.0303
NOW: 0.0303
TMO: 0.0303
CVX: 0.0303
ANET: 0.0303
NVDA: 0.0303
GE: 0.0303
GILD: 0.0303
MU: 0.0303
CMCSA: 0.0303
DIS: 0.0303

Risk Parity for Portfolio B 
Risk Parity Weights (based on ES):
AXP: 0.0303
HON: 0.0303
META: 0.0303
NFLX: 0.0303
PGR: 0.0303
LLY: 0.0303
JPM: 0.0303
VRTX: 0.0303
TJX: 0.0303
EQIX: 0.0303
AAPL: 0.0303
FI: 0.0303
DE: 0.0303
SBUX: 0.0303
GOOGL: 0.0303
T: 0.0303
ABT: 0.0303
BMY: 0.0303
MS: 0.0303
CRM: 0.0303
PFE: 0.0303
SPGI: 0.0303
BRK-B: 0.0303
ADBE: 0.0303
ACN: 0.0303
AMGN: 0.0303
LIN: 0.0303
V: 0.0303
WMT: 0.0303
AMAT: 0.0303
CAT: 0.0303
RTX: 0.0303
UNP: 0.0303

Risk Parity for Portfolio C 
Risk Parity Weights (based on ES):
IBM: 

In [75]:
print("\nAttribution using ES-based Risk Parity Portfolios\n")

for name in ['A', 'B', 'C']:
    print(f"Portfolio {name}")

    sub_pf = portfolio[portfolio['Portfolio'] == name]
    symbols = sub_pf['Symbol'].tolist()
    
    # Get corresponding betas
    pf_betas = np.array([betas_dict[s] for s in symbols])
    
    # Get 2024 returns
    stock_returns = realized_returns[symbols]
    
    # Get SPY returns
    spy_returns = realized_spy.copy()
    
    # Get risk parity weights
    weights_dict = risk_parity_weights[name]
    weights_vec = np.array([weights_dict[s] for s in symbols])
    
    # Run attribution
    attrib_result = expost_factor(weights_vec, stock_returns, spy_returns, pf_betas)
    print(attrib_result['attribution'], "\n")



Attribution using ES-based Risk Parity Portfolios

Portfolio A
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.032882   0.229236
1  Return Attribution  0.265584 -0.036348   0.229236
2     Vol Attribution  0.007683  0.000433   0.008116 

Portfolio B
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.014831   0.255865
1  Return Attribution  0.238187  0.017678   0.255865
2     Vol Attribution  0.006423  0.000386   0.006809 

Portfolio C
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.096561   0.397244
1  Return Attribution  0.287391  0.109853   0.397244
2     Vol Attribution  0.007809  0.000980   0.008789 

